In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드|
load_dotenv()

True

In [2]:
import os

# LangSmith 추적 비활성화
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = ""

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("./dataset-sample-1-5.pdf")

# 문서 로딩
docs = loader.load()

# 문서의 페이지수
len(docs)

5

In [5]:
docs[0].metadata

{'source': './dataset-sample-1-5.pdf',
 'file_path': './dataset-sample-1-5.pdf',
 'page': 0,
 'total_pages': 5,
 'Producer': 'iLovePDF',
 'ModDate': 'D:20250206081020Z'}

In [6]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

In [9]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

In [10]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.1, reasoning: 0.3, multi_context: 0.3, conditional: 0.3}

In [11]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

Generating:   0%|          | 0/10 [00:00<?, ?it/s]              [ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Guernica', 'Pablo Picasso', 'Cubism', "Les Demoiselles d'Avignon", 'Georges Braque']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Georges Braque', 'Paul Cézanne', "Les Demoiselles d'Avignon", 'Cubism', 'Artistic movement']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Guernica', 'Pablo Picasso', 'Cubism', "Les Demoiselles d'Avignon", 'Georges Braque']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.25}
[rag

In [12]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
df = testset.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What significance does 'Les Demoiselles d'Avig...,"[주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Céza...",The answer to given question is not present in...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
1,What movements shaped Cubism and its impact on...,[• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2\...,The context does not provide specific movement...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
2,What linked 'Faune' to Antibes and Picasso in ...,[〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료의...,The context does not provide information linki...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
3,What events in Picasso's life influenced Guern...,[1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새...,The context does not provide specific events i...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
4,How does 'Les Demoiselles d'Avignon' influence...,[• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2\...,The answer to given question is not present in...,multi_context,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
5,What happened with 'Faune' in 1946 regarding i...,[〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료의...,"'Faune' was exhibited in 1946 at Antibes, wher...",multi_context,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
6,What key innovations and themes did Picasso br...,[1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새...,Pablo Picasso brought several key innovations ...,multi_context,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
7,How does 'Guernica' shape our view of Picasso'...,"[우울, 내면의 고통을 청색의 단색조로 형상화하던 청색 시대(1901–1904)와 ...",The answer to given question is not present in...,conditional,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
8,What traits of Cubism influence modern art?,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",The context does not provide specific traits o...,conditional,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
9,"How did Málaga influence Picasso's early art, ...","[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",The answer to given question is not present in...,conditional,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True


In [13]:
# DataFrame을 CSV 파일로 저장
df.to_csv("./gpt_qa_0206.csv", index=False)

In [3]:
import pandas as pd

df = pd.read_csv('dataset_csv/gpt_qa_0206.csv')

In [4]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [5]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


In [6]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,What significance does 'Les Demoiselles d'Avig...,"[""주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Céz...",The answer to given question is not present in...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,현대 미술의 맥락에서 '레 드무아젤 다비뇽'은 어떤 의미를 지니고 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,What movements shaped Cubism and its impact on...,"[""• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2...",The context does not provide specific movement...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,큐비즘을 형성한 사조와 예술에 미친 영향은 무엇인가요?,이 문맥은 큐비즘을 형성한 구체적인 운동이나 예술에 미친 영향을 제공하지 않습니다.
2,What linked 'Faune' to Antibes and Picasso in ...,['〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료...,The context does not provide information linki...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,46년 '포네'와 앙티브와 피카소를 연결한 것은 무엇일까요?,문맥상 'Faune'과 46년 앙티브 및 피카소를 연결하는 정보는 제공되지 않습니다.
3,What events in Picasso's life influenced Guern...,"[""1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 ...",The context does not provide specific events i...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,피카소의 삶에서 게르니카에 영향을 준 사건과 그 영향은 무엇인가요?,피카소의 생애에서 게르니카에 영향을 준 구체적인 사건이나 그 영향에 대한 문맥은 제...
4,How does 'Les Demoiselles d'Avignon' influence...,"[""• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2...",The answer to given question is not present in...,multi_context,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,'레 드무아젤 다비뇽'이 피카소의 스타일 발전에 어떤 영향을 미쳤나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [7]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"[""주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Céz...",simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,현대 미술의 맥락에서 '레 드무아젤 다비뇽'은 어떤 의미를 지니고 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,"[""• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2...",reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,큐비즘을 형성한 사조와 예술에 미친 영향은 무엇인가요?,이 문맥은 큐비즘을 형성한 구체적인 운동이나 예술에 미친 영향을 제공하지 않습니다.
2,['〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료...,reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,46년 '포네'와 앙티브와 피카소를 연결한 것은 무엇일까요?,문맥상 'Faune'과 46년 앙티브 및 피카소를 연결하는 정보는 제공되지 않습니다.
3,"[""1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 ...",reasoning,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,피카소의 삶에서 게르니카에 영향을 준 사건과 그 영향은 무엇인가요?,피카소의 생애에서 게르니카에 영향을 준 구체적인 사건이나 그 영향에 대한 문맥은 제...
4,"[""• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2...",multi_context,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,'레 드무아젤 다비뇽'이 피카소의 스타일 발전에 어떤 영향을 미쳤나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [8]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("dataset_csv/gpt_qa_translate_0206.csv", index=False)